In [1]:
import pickle
import numpy as np
from sklearn.preprocessing import OneHotEncoder
X,y_hr,y_participant,y_activity = pickle.load(open('../data/heart_rate_tabular_data_ppg_dalia.p','rb'))
activity_dict1 = {'No Label':-1,'Sitting':0,'Stairs':1,'Soccer':2,
                'Cycling':3,'Driving':4,'Lunch':-1,'Walking':5,
                'Working':-1}
X = X[:,:,1:].reshape(-1,3,256)
y_activity = np.array([activity_dict1[a] for a in y_activity])
y_activity = OneHotEncoder().fit_transform(y_activity.reshape(-1,1)).todense()

In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import grad
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torchvision import datasets, transforms
import numpy as np
from matplotlib.patches import Rectangle
from torchsummary import summary
import pylab as plt
batch_size = 100
n_epochs = 300
n_classes = 6
latent_size = 100
use_cuda = torch.cuda.is_available()
device = torch.device('cuda' if use_cuda else 'cpu')

In [3]:
from torch.utils.data import TensorDataset, DataLoader
tensor_x = torch.Tensor(X) # transform to torch tensor
tensor_y = torch.Tensor(y_activity)
dataset = TensorDataset(tensor_x,tensor_y) # create your datset
dataloader = DataLoader(dataset,batch_size=batch_size,shuffle=True)


def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        m.weight.data.normal_(0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)

In [4]:
class D(nn.Module):
    def __init__(self, num_classes=6,in_channels=3,
                 out_channels = [50,100,200,50],kernel_size=10,
                 pool_size=2):
        super(D, self).__init__()
        self.ngpu = torch.cuda.device_count()
        self.in_channels = in_channels
        self.out_channels = out_channels

        # Convolution 1
        self.conv1 = nn.Sequential(
            nn.Conv1d(self.in_channels,self.out_channels[0], kernel_size, 1, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.2, inplace=False),
            nn.MaxPool1d(pool_size*2,pool_size,1)
        )
        # Convolution 2
        self.conv2 = nn.Sequential(
            nn.Conv1d(self.out_channels[0],self.out_channels[1],kernel_size, 1, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.2, inplace=False),
            nn.MaxPool1d(pool_size*2,pool_size,1)
        )
        # Convolution 3
        self.conv3 = nn.Sequential(
            nn.Conv1d(self.out_channels[1],self.out_channels[2],kernel_size, 1, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.2, inplace=False),
            nn.MaxPool1d(pool_size*2,pool_size,1)
        )
        # Convolution 4
        self.conv4 = nn.Sequential(
            nn.Conv1d(self.out_channels[2],self.out_channels[3],kernel_size, 1, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.2, inplace=False),
            nn.MaxPool1d(pool_size*2,pool_size,1)
        )
        
        self.flattten = nn.Flatten()
        self.discriminatior_output = nn.Sequential(
            nn.Linear(450,200),
            nn.ReLU(),
            nn.Linear(200,20),
            nn.ReLU(),
            nn.Linear(20,1),
            nn.Sigmoid()            
        )
        self.auxillary_output = nn.Sequential(
            nn.Linear(450,200),
            nn.ReLU(),
            nn.Linear(200,20),
            nn.ReLU(),
            nn.Linear(20,num_classes),
            nn.Softmax()           
        )
        
        
    def forward(self, data):
        data = self.conv1(data)
        data = self.conv2(data)
        data = self.conv3(data)
        data = self.conv4(data)
        data = self.flattten(data)
        real_fake = self.discriminatior_output(data).view(-1, 1).squeeze(1)
        classes = self.auxillary_output(data)
            
        return real_fake,classes



In [5]:
class G(nn.Module):
    def __init__(self,
                 latent_size = 100,
                 num_classes = 6,
                 out_channels = 3,
                 out_timesteps = 256):
        super(G, self).__init__()
        self.ngpu = torch.cuda.device_count()
        self.latent_size = latent_size
        self.out_channels = out_channels
        self.out_timesteps = out_timesteps
        self.latent_size = latent_size
        self.num_classes = num_classes
        self.label_to_noise = nn.Sequential(
            nn.Linear(self.num_classes,self.latent_size//4),
            nn.ReLU(),
            nn.Linear(self.latent_size//4,self.latent_size//2),
            nn.ReLU(),
            nn.Linear(self.latent_size//2,self.latent_size)
        )
        self.flatten = nn.Flatten()
        self.dense_network = nn.Sequential(
            nn.Linear(self.latent_size*2,self.latent_size*4),
            nn.ReLU(),
            nn.Linear(self.latent_size*4,self.latent_size*8),
            nn.Tanh(),
            nn.Linear(self.latent_size*8,self.out_channels*self.out_timesteps)            
        )
        
        
    def forward(self, labels,noise):
        labels = labels.view(-1,self.num_classes)
        noise1 = self.label_to_noise(labels).view(-1,self.latent_size,1)
        noise = noise.view(-1,self.latent_size,1)
        noise = torch.cat((noise,noise1),-1).view(-1,self.latent_size,2)
        noise = self.flatten(noise)
        noise = self.dense_network(noise).view(-1,self.out_channels,self.out_timesteps)
        print(noise.shape)
        return noise



In [6]:
dmodel = D().to(device)
gmodel = G().to(device)
data_noise = torch.empty(batch_size, latent_size, device=device)
print(summary(gmodel,[(1,6),(1,100)]))
print(summary(dmodel,(3,256)))

torch.Size([2, 3, 256])
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                   [-1, 25]             175
              ReLU-2                   [-1, 25]               0
            Linear-3                   [-1, 50]           1,300
              ReLU-4                   [-1, 50]               0
            Linear-5                  [-1, 100]           5,100
           Flatten-6                  [-1, 200]               0
            Linear-7                  [-1, 400]          80,400
              ReLU-8                  [-1, 400]               0
            Linear-9                  [-1, 800]         320,800
             Tanh-10                  [-1, 800]               0
           Linear-11                  [-1, 768]         615,168
Total params: 1,022,943
Trainable params: 1,022,943
Non-trainable params: 0
----------------------------------------------------------------
In

/home/azim/miniconda3/envs/test1/lib/python3.8/site-packages/torch/nn/modules/container.py:117: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


tensor([[-0.1694,  2.9595, -0.3093,  ..., -1.3859,  0.5841,  0.4916],
        [ 0.7396, -0.9584,  1.1998,  ..., -0.3093, -0.9942, -0.4272],
        [-0.1712,  0.5136, -1.0470,  ..., -0.0748,  0.8148, -1.1461],
        ...,
        [ 0.0120, -0.0710,  0.5479,  ..., -0.9409, -0.3572,  0.8597],
        [-0.6438,  0.4561, -0.1333,  ...,  0.6987,  0.8947, -3.7530],
        [ 2.0791, -0.4679,  1.6314,  ...,  1.2703, -1.3746, -1.6066]],
       device='cuda:0')